In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
## import the raw dataset
df = pd.read_csv('TAPAS_flwsht_data.csv')
df

In [ ]:
df['disp_name'].value_counts()

In [ ]:
df[['patient_id', 'encounter_id']].groupby('patient_id').nunique().sort_values(by='encounter_id', ascending=False).head(10)

In [ ]:
## change datetime type
df['recorded_time'] = pd.to_datetime(df['recorded_time'])
df.dtypes

In [ ]:
## convert the weight from ounce pound
for i in range(df.shape[0]):
    if df.iloc[i,3] == 'Weight':
        df.iloc[i,4] = float(df.iloc[i,4])*0.0625

In [ ]:
## check patient and their visit information
df['encounter_id'].value_counts()
df['patient_id'].value_counts()

In [ ]:
## check the mean interval time for each vital signs
print("BP's mean time interval: "+str(df[df['disp_name']=='BP']['diff'].dropna().mean()))
print("SpO2's mean time interval: "+str(df[df['disp_name']=='SpO2']['diff'].dropna().mean()))
print("Height's mean time interval: "+str(df[df['disp_name']=='Height']['diff'].dropna().mean()))
print("Weight's mean time interval: "+str(df[df['disp_name']=='Weight']['diff'].dropna().mean()))
print("Temp's mean time interval: "+str(df[df['disp_name']=='Temp']['diff'].dropna().mean()))
print("Pulse's mean time interval: "+str(df[df['disp_name']=='Pulse']['diff'].dropna().mean()))
print("Heart Rate's mean time interval: "+str(df[df['disp_name']=='Heart Rate']['diff'].dropna().mean()))
print("MAP's mean time interval: "+str(df[df['disp_name']=="MAP (mmHg)"]['diff'].dropna().mean()))
print("Flow Delivered's mean time interval: "+str(df[df['disp_name']=="Flow Delivered (L/min)"]['diff'].dropna().mean()))

## Only keep last value within 1 mins

In [ ]:
df['diff'] = df.sort_values(['encounter_id', 'disp_name', 'recorded_time']
                           ).groupby(['encounter_id','disp_name'])['recorded_time'].diff()
df['last_time'] = df.sort_values(['encounter_id', 'disp_name', 'recorded_time']
                           ).groupby(['encounter_id','disp_name'])['recorded_time'].shift()

In [ ]:
value_to_keep = df[df['diff']<pd.Timedelta(minutes=1)]
value_to_keep

In [ ]:
filtered = value_to_keep[['encounter_id','last_time','disp_name']]
filtered

In [ ]:
i = 0
filtered_df = df.loc[:, ~df.columns.isin(['diff', 'last_time'])]
while i < filtered.shape[0]:
    filtered_df = filtered_df[(filtered_df['encounter_id']!=filtered.iloc[i,0]) | 
                              (filtered_df['recorded_time']!=filtered.iloc[i,1]) | 
                              (filtered_df['disp_name']!=filtered.iloc[i,2])]
    i += 1

## Reshape raw dataset from long format into wide sqa format

In [ ]:
reshaped_df = filtered_df.groupby(["encounter_id","patient_id","recorded_time","disp_name"])[
    'meas_value'].aggregate(lambda x: x).unstack().reset_index()

In [ ]:
reshaped = pd.read_csv('reshaped.csv',index=False)

## Check mean interval time between measurements for each vital signs

### Blood Pressure

In [ ]:
bp = reshaped[['encounter_id', 'recorded_time', 'BP']].dropna()
bp['recorded_time'] = pd.to_datetime(bp['recorded_time'])
bp['time'] = bp['recorded_time'].dt.time
bp['diff'] = bp.groupby('encounter_id')['recorded_time'].diff()

In [ ]:
bp_density = bp[['encounter_id', 'diff']].groupby('encounter_id').mean()['diff'].dt.seconds/3600

In [ ]:
bp_density.reset_index()['diff'].plot(kind='density')

In [ ]:
bp['diff'].mean()

### Heart Rate

In [ ]:
hr = reshaped[['encounter_id', 'recorded_time', 'Heart Rate']].dropna()
hr['recorded_time'] = pd.to_datetime(hr['recorded_time'])
hr['time'] = hr['recorded_time'].dt.time
hr['diff'] = hr.groupby('encounter_id')['recorded_time'].diff()

hr_density = hr[['encounter_id', 'diff']].groupby('encounter_id').mean()['diff'].dt.seconds/3600
hr_density.reset_index()['diff'].plot(kind='density')

In [ ]:
hr['diff'].mean()

### Pulse Rate

In [ ]:
pulse = reshaped[['encounter_id', 'recorded_time', 'Pulse']].dropna()
pulse['recorded_time'] = pd.to_datetime(pulse['recorded_time'])
pulse['time'] = pulse['recorded_time'].dt.time
pulse['diff'] = pulse.groupby('encounter_id')['recorded_time'].diff()
pulse_density = pulse[['encounter_id', 'diff']].groupby('encounter_id').mean()['diff'].dt.seconds/3600
pulse_density.reset_index()['diff'].plot(kind='density')

In [ ]:
pulse['diff'].mean()

### MAP (mmHg)

In [ ]:
maps = reshaped[['encounter_id', 'recorded_time', 'MAP (mmHg)']].dropna()
maps['recorded_time'] = pd.to_datetime(maps['recorded_time'])
maps['time'] = maps['recorded_time'].dt.time
maps['diff'] = maps.groupby('encounter_id')['recorded_time'].diff()
map_density = maps[['encounter_id', 'diff']].groupby('encounter_id').mean()['diff'].dt.seconds/3600
map_density.reset_index()['diff'].plot(kind='density')

In [ ]:
maps['diff'].mean()

### Respiration

In [ ]:
resp = reshaped[['encounter_id', 'recorded_time', 'Resp']].dropna()
resp['recorded_time'] = pd.to_datetime(resp['recorded_time'])
resp['time'] = resp['recorded_time'].dt.time
resp['diff'] = resp.groupby('encounter_id')['recorded_time'].diff()
resp_density = resp[['encounter_id', 'diff']].groupby('encounter_id').mean()['diff'].dt.seconds/3600
resp_density.reset_index()['diff'].plot(kind='density')

In [ ]:
resp['diff'].mean()

### SpO2

In [ ]:
sp = reshaped[['encounter_id', 'recorded_time', 'SpO2']].dropna()
sp['recorded_time'] = pd.to_datetime(sp['recorded_time'])
sp['time'] = sp['recorded_time'].dt.time
sp['diff'] = sp.groupby('encounter_id')['recorded_time'].diff()
sp_density = sp[['encounter_id', 'diff']].groupby('encounter_id').mean()['diff'].dt.seconds/3600
sp_density.reset_index()['diff'].plot(kind='density')

In [ ]:
sp['diff'].mean()

### Temperature

In [ ]:
temp = reshaped[['encounter_id', 'recorded_time', 'Temp']].dropna()
temp['recorded_time'] = pd.to_datetime(temp['recorded_time'])
temp['time'] = temp['recorded_time'].dt.time
temp['diff'] = temp.groupby('encounter_id')['recorded_time'].diff()
temp_density = temp[['encounter_id', 'diff']].groupby('encounter_id').mean()['diff'].dt.seconds/3600
temp_density.reset_index()['diff'].plot(kind='density')

In [ ]:
temp['diff'].mean()

### Weighted mean time interval


In [ ]:
weighted_mean =  (len(bp)*bp['diff'].mean() + len(temp)*temp['diff'].mean() + 
                  len(sp)*sp['diff'].mean() + len(resp)*resp['diff'].mean() +
                  len(maps)*maps['diff'].mean() + len(hr)*hr['diff'].mean() +
                  len(pulse)*pulse['diff'].mean())/(len(bp)+len(temp)+len(sp)+len(resp)+len(maps)+len(hr)+len(pulse))
weighted_mean
## 2 hours

### plot all vital signs

In [ ]:
temp['temp_diff'] = temp['diff']
sp['sp_diff'] = sp['diff']
resp['resp_diff'] = resp['diff']
maps['map_diff'] = maps['diff']
pulse['pulse_diff'] = pulse['diff']
hr['hr_diff'] = hr['diff']
bp['bp_diff'] = bp['diff']

temp_diff = temp[['encounter_id','temp_diff']].groupby('encounter_id').mean().reset_index()
sp_diff = sp[['encounter_id','sp_diff']].groupby('encounter_id').mean().reset_index()
resp_diff = resp[['encounter_id','resp_diff']].groupby('encounter_id').mean().reset_index()
maps_diff = maps[['encounter_id','map_diff']].groupby('encounter_id').mean().reset_index()
pulse_diff = pulse[['encounter_id','pulse_diff']].groupby('encounter_id').mean().reset_index()
hr_diff = hr[['encounter_id','hr_diff']].groupby('encounter_id').mean().reset_index()
bp_diff = bp[['encounter_id','bp_diff']].groupby('encounter_id').mean().reset_index()

In [ ]:
diff = pd.merge(pd.merge(pd.merge(temp_diff,pulse_diff,on='encounter_id'), 
                         pd.merge(sp_diff,hr_diff,on='encounter_id'),on='encounter_id'), 
                pd.merge(pd.merge(resp_diff, bp_diff, on='encounter_id'),maps_diff,on='encounter_id'))

In [ ]:
diff = diff.apply(lambda x: x.dt.seconds/3600 if x.name in ['temp_diff', 'pulse_diff', 'sp_diff', 'hr_diff', 'resp_diff',
                                                'bp_diff', 'map_diff'] else x)

In [ ]:
diff.iloc[:,1:].plot(figsize = (14, 7),kind='density',xticks=[i for i in range(25)])

## Plot the vital sign time distribution

In [ ]:
bp['time_dist'] = bp['time'].apply(lambda x: x.hour+x.minute/100)
hr['time_dist'] = hr['time'].apply(lambda x: x.hour+x.minute/100)
pulse['time_dist'] = pulse['time'].apply(lambda x: x.hour+x.minute/100)
maps['time_dist'] = maps['time'].apply(lambda x: x.hour+x.minute/100)
resp['time_dist'] = resp['time'].apply(lambda x: x.hour+x.minute/100)
sp['time_dist'] = sp['time'].apply(lambda x: x.hour+x.minute/100)
temp['time_dist'] = temp['time'].apply(lambda x: x.hour+x.minute/100)

In [ ]:
## trying to see if the time interval between each measure is even distributed with time or not. 
## ex: maybe only measure the value at daytime. 

emp['time_dist'].plot(kind='density')
bp['time_dist'].plot(kind='density')
hr['time_dist'].plot(kind='density')
pulse['time_dist'].plot(kind='density')
maps['time_dist'].plot(kind='density')
sp['time_dist'].plot(kind='density')
resp['time_dist'].plot(kind='density')

## Slide data & fill the missing value

In [ ]:
reshaped['id_transform'] = reshaped.groupby(['encounter_id','patient_id']).ngroup()+1

In [ ]:
map_index = reshaped[['encounter_id','patient_id','id_transform']]
map_index.to_csv('map_index.csv',index=False)

In [ ]:
# split by id
split = reshaped

In [ ]:
# fill the missing value by previous value, then fill the missing value by last value

fill_missing = split.groupby(['encounter_id','patient_id']).ffill().bfill(axis='rows').reset_index().iloc[:,1:]

In [ ]:
fill_missing['id_transform'].value_counts().sort_values().hist(bins=200)

In [ ]:
no_duplicate = fill_missing.drop_duplicates(subset=['BP', 'Flow Delivered (L/min)', 'Heart Rate', 'Height',
                                                    'MAP (mmHg)', 'Pulse', 'Resp', 'SpO2', 'Temp', 'Weight'])
no_duplicate.shape

In [ ]:
no_duplicate['recorded_time'] = pd.to_datetime(no_duplicate['recorded_time'])

In [ ]:
# 
df_2 = no_duplicate.copy()

df_2['diff'] = df_2.groupby('id_transform')['recorded_time'].diff()

In [ ]:
## Only keep the first value within 2-hour. Delete the duplicate while still capture the detail of the data.

df_3 = df_2.groupby(['id_transform',df_2.recorded_time.dt.floor('2H')]).first()
print(df_2.shape)
print(df_2.groupby(['id_transform',df_2.recorded_time.dt.floor('2H')]).first().shape)

In [ ]:
df_4 = df_3.reset_index(level=1, drop=True).reset_index()

In [ ]:
df_4['id_transform'].value_counts().sort_values().hist(bins=200)

In [ ]:
original_count = no_duplicate['id_transform'].value_counts().reset_index()
filtered_count = df_4['id_transform'].value_counts().reset_index()

In [ ]:
count_comparison = filtered_count.merge(original_count,on = 'index')

In [ ]:
count_comparison.columns = ['encounter_id','filtered_count', 'original_count']

In [ ]:
count_comparison['dropped_ratio'] = round((count_comparison['original_count']-count_comparison['filtered_count'])
                                          /count_comparison['original_count'], 2)

In [ ]:
count_comparison.describe()

## Slice the dataset by their encounter id.

## prepare for fine-tune

In [ ]:
for i in range(1,1001):
    encounter = df_4[df_4['id_transform']==i].drop(['id_transform','diff'],axis=1)
    
    if encounter.shape[0] <=10:
       # encounter.insert(0,'Pos',[pos for pos in range(1,encounter.shape[0]+1)])
        encounter.to_csv(f'dataset/{i}.csv',index=False)
    else:
        add_pos = encounter.tail(10)
          #  pos = [i for i in range(1,len(encounter[50*(j-1):50*j])+1)]
          #  add_pos.insert(0, 'Pos',pos)
        add_pos.to_csv(f'dataset/{i}.csv',index=False)  